In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
import os
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 21.7 ms, sys: 7.82 ms, total: 29.5 ms
Wall time: 191 ms


### equivalent reflectivity factor

In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

### convert equivalent reflectivity factor to Iris cube

In [4]:
%%time
ERF = equivalent_reflectivity_factor.to_iris()
ERF

CPU times: user 3.96 s, sys: 562 ms, total: 4.53 s
Wall time: 4.97 s


Equivalent Reflectivity Factor (dBZ),time,projection_y_coordinate,projection_x_coordinate
Shape,301,400,400
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
latitude,-,x,x
longitude,-,x,x


In [5]:
%%time
# Determine temporal and spatial sampling of the input data:
#grid_spacing = 1km... but tobac uses meters... 1000m = 1km
#time_spacing = 5 minutes time resolution... tobac uses seconds..... 
#since our time_spacing is 5 min, we get our time spacing in seconds.. if 60 sec = 1 min? then 5 mins = 300s... so time-spacing is 300
dxy,dt=tobac.utils.get_spacings(ERF, grid_spacing=1000, time_spacing=300)  #tobac detect it by default
dxy, dt 

CPU times: user 51 µs, sys: 8 µs, total: 59 µs
Wall time: 61.5 µs


(1000, 300)

# DETECTION FEATURE

In [6]:
%%time
# threshold = np.arange(5, 20.1, 1)
threshold = [15]
parameters_features = {}
parameters_features['target'] = 'maximum'
parameters_features['threshold'] = threshold
parameters_features['n_min_threshold'] = 0 #set to zero or one always; 
parameters_features['n_erosion_threshold'] = 0 #another filtering/smoothing method.
parameters_features['position_threshold'] ='weighted_diff'
parameters_features['sigma_threshold'] = 1 #smoothing data
# parameters_features['min_distance'] = 15

# Using 'center' here outputs the feature location as the arithmetic center of the detected feature
Features = tobac.feature_detection_multithreshold(field_in=ERF, dxy=dxy, **parameters_features)

CPU times: user 4.38 s, sys: 19.7 ms, total: 4.4 s
Wall time: 4.95 s


In [7]:
%%time
Features.head()

CPU times: user 92 µs, sys: 9 µs, total: 101 µs
Wall time: 103 µs


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude
0,0,1,110.987364,214.505783,38,15,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.170987e+06,131505.783191,70.424947,13.466409
1,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355
2,0,3,134.112530,398.419942,11,15,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.194113e+06,315419.941581,70.045530,18.180645
3,0,4,138.283196,209.774751,22,15,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198283e+06,126774.750896,70.179600,13.300586
4,0,5,143.000000,384.000000,1,15,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.203000e+06,301000.000000,69.983643,17.780252


In [9]:
# Features.to_csv('../saved-files/threshold-15/Features-15.csv', index=False)

# SEGMENTATION

In [10]:
%%time
# Keyword arguments for the segmentation step:
parameters_segmentation={}
parameters_segmentation['target']='maximum'
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']=15

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [11]:
%%time
# Perform segmentation and save results to files:
Mask_ERF, Features_ERF = tobac.segmentation_2D(Features,ERF,dxy,**parameters_segmentation)

CPU times: user 10.3 s, sys: 302 ms, total: 10.6 s
Wall time: 11.1 s


In [12]:
type(Mask_ERF)

iris.cube.Cube

In [13]:
iris.save(Mask_ERF, '../saved-files/threshold-15/Mask_ERF_iris-15.nc')

In [14]:
%%time
Mask_ERF

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


Segmentation Mask (1),time,projection_y_coordinate,projection_x_coordinate
Shape,301,400,400
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
latitude,-,x,x
longitude,-,x,x


In [15]:
%%time
# Convert the segmentation data from iris cube to DataArray
segmented_data = xr.DataArray.from_iris(Mask_ERF)
segmented_data

CPU times: user 2.3 ms, sys: 1.02 ms, total: 3.32 ms
Wall time: 3.32 ms


<xarray.DataArray 'segmentation_mask' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=int32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
    lat      (Yc, Xc) float32 ...
    lon      (Yc, Xc) float32 ...
Attributes:
    long_name:  segmentation_mask

In [16]:
segmented_data.to_netcdf('../saved-files/threshold-15/segmented_ERF-xr-15.nc')

# TRAJECTORY LINKING

In [17]:
%%time
# keyword arguments for linking step
parameters_linking={}
parameters_linking['v_max']=15
parameters_linking['stubs']=1
parameters_linking['order']=1
parameters_linking['extrapolate']=0 
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=15
parameters_linking['method_linking']= 'predict'
# parameters_linking['time_cell_min'] = 10

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.44 µs


In [18]:
%%time
# Track=tobac.linking_trackpy(Features, ERF, dt=dt, dxy=dxy, **parameters_linking)
Track = tobac.linking_trackpy(Features, ERF, dt=dt, dxy=dxy, **parameters_linking)

Frame 300: 35 trajectories present.
CPU times: user 2.51 s, sys: 32.5 ms, total: 2.55 s
Wall time: 3.02 s


In [19]:
Track.to_csv('../saved-files/threshold-15/Track-15.csv', index=False)

In [20]:
# latA = 69.141281 #latitude of COMBLE site
# lonA = 15.684166-1 #longitude of COMBLE site -1

<h1 style="color:red;">TRACKED INFO</h1>

In [21]:
Track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,110.987364,214.505783,38,15,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.170987e+06,131505.783191,70.424947,13.466409,1,0 days 00:00:00
1,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00
2,0,3,134.112530,398.419942,11,15,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.194113e+06,315419.941581,70.045530,18.180645,3,0 days 00:00:00
3,0,4,138.283196,209.774751,22,15,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.198283e+06,126774.750896,70.179600,13.300586,4,0 days 00:00:00
4,0,5,143.000000,384.000000,1,15,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.203000e+06,301000.000000,69.983643,17.780252,5,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23064,300,31,293.118966,382.430192,44,15,23065,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.353119e+06,299430.192009,68.629287,17.251813,6569,0 days 00:20:00
23065,300,32,289.576755,389.502260,4,15,23066,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.349577e+06,306502.259875,68.653092,17.432265,6594,0 days 00:05:00
23066,300,33,295.663385,392.347830,10,15,23067,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.355663e+06,309347.829884,68.594665,17.481323,6595,0 days 00:05:00
23067,300,34,335.000000,248.537852,2,15,23068,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.395000e+06,165537.852380,68.368117,13.953887,6603,0 days 00:00:00


# Sort Tracked info based on cell_id and time

In [22]:
%%time
track = Track.sort_values(['cell', 'time_cell'])
track = track.reset_index(drop=True)
track.head()

CPU times: user 6.16 ms, sys: 2.89 ms, total: 9.05 ms
Wall time: 89.2 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,110.987364,214.505783,38,15,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.170987e+06,131505.783191,70.424947,13.466409,1,0 days 00:00:00
1,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00
2,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00
3,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00
4,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00


In [23]:
track.to_csv('../saved-files/threshold-15/track-reset-15.csv', index=False)

In [24]:
# track = pd.read_csv('saved-files/track-reset.csv')

In [25]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,110.987364,214.505783,38,15,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.170987e+06,131505.783191,70.424947,13.466409,1,0 days 00:00:00
1,0,2,133.891782,247.588144,6,15,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.193892e+06,164588.143590,70.196696,14.290355,2,0 days 00:00:00
2,1,5,135.850880,248.149120,2,15,86,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.195851e+06,165149.120397,70.178537,14.301097,2,0 days 00:05:00
3,2,5,139.000000,249.440269,2,15,169,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.199000e+06,166440.268555,70.149081,14.328411,2,0 days 00:10:00
4,3,4,143.476756,250.870342,6,15,251,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.203477e+06,167870.342039,70.107474,14.356623,2,0 days 00:15:00


<h1 style="color:red;  text-align: center;">END OF TRACK</h1>